<a href="https://colab.research.google.com/github/playbase/ETL_Hive_Assignments/blob/main/CNN_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-samples:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5136555%2F8605872%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240611%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240611T154208Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D839378d351782c946209f62134fdec135afce48ab1a7200359675f327d0bcac9386a2b5e739e8f533597b870f9d3b65e05b0f8e0d06d16472f994829110e76402f8c2923f2299ac35ef330f572ed16b514a02eafc5461a1435fa2b2fb19613d175bd79a21f535eb4195a9496e2c1889616952b2bc5d742ff2d59ed3da0ce9674a4b195f873edb4e9d7e9d6b8303b7af9a2040d0c2b6c9428a9f9cf5991868527db28554498d0f7e61bd290e6e4a082a6666e91f7fcc0626a89e3e3403f82b9f36780475d9db49f81289b0f23ed0b723aa73f1bfe5145502a9ddae6dd128d2c3b1bf9f4402bf80132c77e7823d91f7ea365bfb51510909cc4ee8582461e688660,fashion-mnist:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5169420%2F8633094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240611%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240611T154208Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9cd6e5caafc177f2b2d0457ac8a44370c6673973c7c881e0c9f0ce4609dd12a1c2d7f46bb2cb76d1302f3a2204ead9dbdf38ce3650e6dfbdb17dc88789b68f2e0b7cafa3a51ac8bcc5da5f6972cacb969ac3ab21875d95faa6f42fd2b34133e3c0fc26cc6914e603451984a9abcf8a47564adc4c991987e06b87403de960d5cc2920d0722345917d9400dcb5550331e016285035f6aaaa509fabdece90da9f0c28a4c961c5094ecad88208cff78b75eb6af13b94033b262cb6a0f1d498881c2940a665117d1ee0c4d3f720b8089aff9f7a60ae95e002d801ce3d9991ffffc452bd57b48a3da6c700b60f6882c6f3a2062e944aa747a35bf5a26b1cf3742022f9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install keras
!pip install visualkeras

In [ ]:
#All imports here
import cv2
import pandas as pd
import io
import visualkeras
import os
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import visualkeras
from keras.utils import plot_model
import math

In [ ]:
#Data Collection
from keras.datasets import fashion_mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [ ]:
#Image Visualisation
plt.imshow(x_train[100],plt.get_cmap('binary'))

In [ ]:
y_train[20000]

In [ ]:
x=1
for i in range(25,50):
    plt.subplot(5,5,x)
    plt.imshow(x_train[i],plt.get_cmap('binary'))
    x+=1

In [ ]:
#Encoding
from keras.utils import to_categorical
y_train1=to_categorical(y_train)
y_test1=to_categorical(y_test)

In [ ]:
#Build Model
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Convolution2D,MaxPooling2D,Flatten
nn=Sequential()

In [ ]:
#16 forms of the same image
nn.add(Convolution2D(filters=16,kernel_size=(3,3),input_shape=(28,28,1)))
nn.add(MaxPooling2D(pool_size=(2,2)))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

nn.add(Convolution2D(filters=16,kernel_size=(3,3)))
nn.add(MaxPooling2D(pool_size=(2,2)))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

nn.add(Convolution2D(filters=16,kernel_size=(3,3)))
nn.add(MaxPooling2D(pool_size=(2,2)))
nn.add(BatchNormalization())
nn.add(Dropout(0.1))

nn.add(Flatten())

nn.add(Dense(1000,activation='relu'))
nn.add(Dense(1000,activation='relu'))
nn.add(Dense(1000))

nn.add(Dense(10,activation='softmax'))

In [ ]:
#Compile
nn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#Add Callbacks
from keras.callbacks import EarlyStopping
Early=EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
visualkeras.layered_view(nn)

In [ ]:
#Fit the model
hist=nn.fit(x_train,y_train1,validation_split=0.2,callbacks=[Early],epochs=5)

In [ ]:
#Visulaise the result
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])

In [ ]:
#Accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
# Evaluation Of Training data
from sklearn.metrics import accuracy_score,classification_report,multilabel_confusion_matrix
import numpy as np

In [ ]:
y_pred_train=nn.predict(x_train)
y_pred_train[0]

In [ ]:
y_pred_train1=np.argmax(y_pred_train,axis=-1)
y_pred_train1

In [ ]:
y_train

In [ ]:
#Training Accuracy
acc=accuracy_score(y_pred_train1,y_train)
clf=classification_report(y_pred_train1,y_train)
cnf=multilabel_confusion_matrix(y_pred_train1,y_train)

print('Accuracy:',acc)
print('Confusion matrix:\n',cnf)
print('classification Report:\n',clf)

In [ ]:
ts_pred=nn.predict(x_test)
ts_pred

In [ ]:
y_pred_test=np.argmax(ts_pred,axis=-1)
y_pred_test

In [ ]:
y_test

In [ ]:
#Testing Accuracy
acc1=accuracy_score(y_pred_test,y_test)
clf1=classification_report(y_pred_test,y_test)
cnf1=multilabel_confusion_matrix(y_pred_test,y_test)

print('Accuracy:',acc1)
print('Confusion matrix:\n',cnf1)
print('classification Report:\n',clf1)

In [ ]:
labels = {0 : "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat",
          5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"}

X_test__ = x_test.reshape(x_test.shape[0], 28, 28)
fig, axis = plt.subplots(4, 4, figsize=(12, 14))
for i, ax in enumerate(axis.flat):
    ax.imshow(x_test[i], cmap='binary')
    ax.set(title = f"Real Class is {labels[y_test[i]]}\nPredict Class is {labels[y_pred_test[i]]}");
